<a href="https://colab.research.google.com/github/MxD-lab/SNN_Simulation/blob/neralnetwork/neralnetwork_pp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
import numpy 

In [ ]:
"""
活性化関数たち
"""
#シグモイド関数
def sigmoid(x):
  return 1/(1+np.exp(-x))

#ソフトマックス関数
def softmax(a):
  c = np.max(a)
  exp_a = np.exp(a-c)#オーバーフロー対策
  sum_exp_a = np.sum(exp_a)
  y = exp_a/sum_exp_a
  return y

In [ ]:
"""
損失関数たち
"""
#2乗和誤差
def mean_squared_error(y,t):
  return 0.5*np.sum((y-t)**2)

#交差エントロピー誤差
def cross_entropy_error(y,t):
  delta = 1e-7
  return -np.sum(t*np.log(y+delta))

In [ ]:
"""
3層ニューラルネットワーク
"""


X = np.array([1.0,0.5])#入力
W1 = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])#1層目の重み
B1 = np.array([0.1,0.2,0.3])#1層目の閾値

A1 = np.dot(X,W1)+B1
Z1 = sigmoid(A1)#1層目活性化関数をかける

W2 = np.array([[0.1,0.4],[0.2,0.5],[0.3,0.6]])#1層目から2層目への重み
B2 = np.array([0.1,0.2])#1層目から2層目の閾値

A2 = np.dot(Z1,W2)+B2
Z2 = sigmoid(A2)#2層目活性化関数をかける

def identity_function(x):#出力層の活性化関数
  return x
  
W3 = np.array([[0.1,0.3],[0.2,0.4]])#2層目から3層目への重み
B3 = np.array([0.1,0.2])#2層目から3層目の閾値

A3 = np.dot(Z2,W3)+B3
Y = identity_function(A3)#出力層活性化関数をかける


In [ ]:
#↑まとめ
def init_network():
  network = {}
  network["W1"] = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])#1層目の重み
  network["b1"] = np.array([0.1,0.2,0.3])#1層目の閾値
  network["W2"] = np.array([[0.1,0.4],[0.2,0.5],[0.3,0.6]])#1層目から2層目への重み
  network["b2"] = np.array([0.1,0.2])#1層目から2層目の閾値
  network["W3"] = np.array([[0.1,0.3],[0.2,0.4]])#2層目から3層目への重み
  network["b3"] = np.array([0.1,0.2])#2層目から3層目の閾値

  return network

def forward(network,x):
  W1,W2,W3 = network["W1"],network["W2"],network["W3"]
  b1,b2,b3 = network["b1"],network["b2"],network["b3"]

  a1 = np.dot(x,W1) + b1
  z1 = sigmoid(a1)
  a2 = np.dot(z1,W2) + b2
  z2 = sigmoid(a2)
  a3 = np.dot(z2,W3) + b3
  y  = identity_function(a3)

  return y

network = init_network()
x = np.array([1.0,0.5])
y = forward(network,x)
print(y)


[0.31682708 0.69627909]


In [ ]:
"""
勾配
"""
def numerical_gradient(f,x):
  h = 1e-4 #0.0001
  grad = np.zeros_like(x) #xと同じ形状の配列を作成

  for idx in range(x.size):
    tmp_val = x[idx]

    #f(x+h)の計算
    x[idx]= tmp_val + h #f(x+h)の計算
    fxh1 = f(x)

    #f(x-h)の計算
    x[idx]= tmp_val - h #f(x-h)の計算
    fxh2 = f(x)

    grad[idx] = (fxh1 - fxh2)/(2*h)
    x[idx] = tmp_val #値を元に戻す

  return grad

  """
勾配化法

init_x:初期値
lr:learning rate(学習率)
step_num:勾配法による繰り返し回数

勾配をnumerical_gradientで求めて、
その勾配に学習率を掛けた値で更新する処理を
step_numで指定された回数繰り返す
"""

def gradient_descent(f,init_x,lr= 0.01,step_num = 100):
  x = init_x

  for i in range(step_num):
    grad = numerical_gradient(f,x)

    x -= lr *grad
  
  return x

In [ ]:
"""
2層ニューラルネットワークの学習
"""
class TwoLayerNet:

  def __init__(self,input_size,hidden_size,output_size,weight_init_std = 0.01):
    
    #重みの初期化
    self.params = {}
    self.params["W1"] = weight_init_std*\
                        np.random.randn(input_size,hidden_size)
    
    self.params["b1"] = np.zeros(hidden_size)

    self.params["W2"] = weight_init_std*\
                        np.random.randn(hidden_size,output_size)
    
    self.params["b2"] = np.zeros(output_size)

  #推論を行う
  def predict(self,x):
    W1,W2 = self.params["W1"],self.params["W2"]
    b1,b2 = self.params["b1"],self.params["b2"]

    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    y = softmax(a2)

    return y

  #ｘ：入力データ、t：教師データ
  def loss(self,x,t):
    y = self.predict(x)

    return cross_entropy_error(y,t)

  #認知精度を求める
  def accuracy(self,x,t):
    y = predict(x)
    #最大値を返す axis:次元
    accuracy = np.sum(y==t)/float(x.shape[0])
    return accuracy

  def numerical_gradient(self,x,t):
    loss_W = lambda W:self.loss(x,t)

    grads = {}#勾配を保持するディクショナリ関数
    grads["W1"] = numerical_gradient(loss_W,self.params["W1"])
    grads["b1"] = numerical_gradient(loss_W,self.params["b1"])
    grads["W2"] = numerical_gradient(loss_W,self.params["W2"])
    grads["b2"] = numerical_gradient(loss_W,self.params["b2"])

    return grads




In [ ]:

net = TwoLayerNet(input_size=784,hidden_size=100,output_size=10)
x= np.random.rand(100,784)#ダミー入力データ
t = np.random.rand(100,10)#ダミーテストデータ
y = net.predict(x)

grads = net.numerical_gradient(x,t)#勾配を計算

IndexError: ignored